In [1]:
import numpy as np
from sklearn.datasets import make_classification, load_iris
from sklearn.model_selection import train_test_split

In [2]:
X, y = make_classification(n_samples=600, n_features=6, n_classes = 2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=1234)

In [3]:
X_train, X_test = X_train.T, X_test.T
y_train = y_train.reshape(1, len(y_train))
y_test = y_test.reshape(1, len(y_test))

In [4]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(6, 480) (1, 480)
(6, 120) (1, 120)


### Logistic Regression

In [5]:
class LogisticRegression:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.W = np.random.randn(1, self.X.shape[0])
        self.b = np.zeros(shape=(1,1))
        
    def _feed_forward(self):
        Z = np.dot(self.W, self.X) + self.b
        return Z
    
    def _sigmoid(self, Z):
        A = 1 / (1 + np.exp(-Z))
        return A
    
    def _sigmoid_derivative(self, A):
        return self._sigmoid(A) * (1-self._sigmoid(A))
    
    def _binary_cross_entropy(self, Y_pred):
        m = Y_pred.shape[1]
        cost = -1 / m * np.sum(self.y * np.log(Y_pred + 1e-8) + (1 - self.y) * (np.log(1 - Y_pred + 1e-8)))
        return cost
    
    def _compute_grads(self, Y_pred):
        '''
        Though it can be done in one shot as:
        dL/dW = (Y-Y_pred) dot X.T
        still, some times results get more accurate by doing this step computation
        '''
        #dL_by_dY_pred = np.divide(self.y, Y_pred) - np.divide(1-self.y, 1-Y_pred)
        #dY_pred_by_dZ = dL_by_dY_pred * Y_pred * (1-Y_pred)
        
        W_grad = np.dot((Y_pred-self.y), self.X.T) #np.dot((self.y-Y_pred), self.X.T)
        b_grad = np.sum(Y_pred-self.y, axis=1, keepdims=True)  #dY_pred_by_dZ
        
        return (W_grad, b_grad)
    
    def accuracy(self, prediction, ground_truth):
        number_example = prediction.shape[1]
        accuracy = (np.sum(np.round(prediction) == ground_truth)) / number_example
        return accuracy
    
    def train(self, learning_rate = 0.01, epochs = 100):
        for epoch in range(epochs):
            Z_pred = self._feed_forward()
            Y_pred = self._sigmoid(Z_pred)
            
            cost = self._binary_cross_entropy(Y_pred)
            W_grad, b_grad = self._compute_grads(Y_pred)
            
            self.W -= learning_rate * W_grad
            self.b -= learning_rate * b_grad
            
            if epoch % 50 == 0:
                print(f"After epoch {epoch} cost: {cost}, acc: {self.accuracy(Y_pred, self.y)}")

In [6]:
p = LogisticRegression(X_train, y_train)

In [7]:
p.train(learning_rate=0.01, epochs=250)

After epoch 0 cost: 0.719276945534596, acc: 0.63125
After epoch 50 cost: 0.33777384661657606, acc: 0.8770833333333333
After epoch 100 cost: 0.33777384661657617, acc: 0.8770833333333333
After epoch 150 cost: 0.33777384661657617, acc: 0.8770833333333333
After epoch 200 cost: 0.33777384661657617, acc: 0.8770833333333333


### SKLEARN

In [8]:
from sklearn.linear_model import LogisticRegression

In [9]:
regressor = LogisticRegression().fit(X_train.T, y_train.T)

/home/anindya/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [10]:
preds = regressor.predict(X_test.T)

In [11]:
preds = preds.reshape(1, len(preds))

In [12]:
p.accuracy(preds, y_test)

0.8